In [ ]:
!pip install googlemaps pandas

In [ ]:
import pandas as pd
import googlemaps
from datetime import datetime
import time

# ============ CONFIGURACIÓN ============

API_KEY = "API_KEY"   # <-- REEMPLAZA ESTO

ORIGINS_CSV_PATH = "/content/input_facilities.xlsx"
DESTINATIONS_CSV_PATH = "/content/raw_pixels.csv"

ORIGIN_LON_COL = "origin_lon"
ORIGIN_LAT_COL = "origin_lat"

DEST_LON_COL = "destino_lon"
DEST_LAT_COL = "destino_lat"

OUTPUT_CSV_PATH = "/content/input_matrix_distance_facilities_pixels.xlsx"

SLEEP_BETWEEN_CALLS = 0.1  # en segundos

# ======================================

gmaps = googlemaps.Client(key=API_KEY)

In [ ]:
def test_google_connection(client: googlemaps.Client) -> None:
    """
    Hace una llamada simple a Distance Matrix para verificar que:
    - La API Key es válida
    - La API está habilitada
    - Hay billing activo
    """
    print("Probando conexión con Google Distance Matrix API...")
    try:
        # Punto dummy cualquiera
        origin = (0.0, 0.0)
        destination = (0.1, 0.1)

        result = client.distance_matrix(
            origins=[origin],
            destinations=[destination],
            mode="driving"
        )

        status_overall = result.get("status", "UNKNOWN")
        element_status = result["rows"][0]["elements"][0].get("status", "UNKNOWN")

        if status_overall == "OK" and element_status == "OK":
            print("✅ Conexión OK: la API está respondiendo correctamente.")
        else:
            print(f"⚠️ Respuesta con problemas. status={status_overall}, element_status={element_status}")
            print("Revisa que la Distance Matrix API esté habilitada y que el billing esté activo.")
    except Exception as e:
        print("❌ Error al probar conexión con Google API:")
        print(e)
        raise

test_google_connection(gmaps)


In [ ]:
# Leer CSV de orígenes y destinos
origins_df = pd.read_excel(ORIGINS_CSV_PATH)[['id_facility', 'lon', 'lat']]
destinations_df = pd.read_csv(DESTINATIONS_CSV_PATH)[['layer','pixel','lon','lat']]

origins_df[ORIGIN_LON_COL] = origins_df['lon']
origins_df[ORIGIN_LAT_COL] = origins_df['lat']

destinations_df[DEST_LON_COL] = destinations_df['lon']
destinations_df[DEST_LAT_COL] = destinations_df['lat']

print(len(origins_df), len(destinations_df))

In [ ]:
# Cross join: todas las combinaciones origen-destino
origins_df["_tmp_key"] = 1
destinations_df["_tmp_key"] = 1

pairs_df = origins_df.merge(destinations_df, on="_tmp_key", how="outer").drop(columns=["_tmp_key"])
print(len(pairs_df))

In [ ]:
def get_distance_info(row, client: googlemaps.Client):
    """
    Dado un row con:
      - origin_lat, origin_lon
      - dest_lat, dest_lon
    consulta Google Distance Matrix y devuelve:
      - distance_m
      - duration_sec
      - duration_in_traffic_sec
    """
    origin = (row[ORIGIN_LAT_COL], row[ORIGIN_LON_COL])
    destination = (row[DEST_LAT_COL], row[DEST_LON_COL])

    try:
        result = client.distance_matrix(
            origins=[origin],
            destinations=[destination],
            mode="driving",
            departure_time=datetime.now(),  # necesario para obtener duration_in_traffic
            traffic_model="best_guess"
        )

        element = result["rows"][0]["elements"][0]

        if element.get("status") != "OK":
            print(f"Fila {row.name}: status={element.get('status')}")
            return pd.Series({
                "distance_m": None,
                "duration_sec": None,
                "duration_in_traffic_sec": None
            })

        distance_m = element["distance"]["value"]                  # en metros
        duration_sec = element["duration"]["value"]                # en segundos
        duration_in_traffic_sec = element.get("duration_in_traffic", {}).get("value")

        # Pausa ligera para no saturar la API
        if SLEEP_BETWEEN_CALLS > 0:
            time.sleep(SLEEP_BETWEEN_CALLS)

        return pd.Series({
            "distance_m": distance_m,
            "duration_sec": duration_sec,
            "duration_in_traffic_sec": duration_in_traffic_sec
        })

    except Exception as e:
        print(f"Error en fila {row.name}: {e}")
        return pd.Series({
            "distance_m": None,
            "duration_sec": None,
            "duration_in_traffic_sec": None
        })


# Aplicar a todas las combinaciones origen-destino
distances = pairs_df.apply(get_distance_info, axis=1, client=gmaps)

# Unimos resultados
pairs_df = pd.concat([pairs_df, distances], axis=1)


In [ ]:
pairs_df

In [ ]:
df = pairs_df[['layer','pixel','id_facility', 'origin_lat', 'destino_lat', 'origin_lon', 'origin_lat', 'distance_m',	'duration_sec','duration_in_traffic_sec']]
df

In [ ]:
# Agregamos columnas en minutos (opcional)
df["distance"] = df["distance_m"] / 1000
df["travel_time"] = df["duration_sec"] / 3600
df["travel_time_in_traffic"] = df["duration_in_traffic_sec"] / 3600

In [ ]:
df

In [ ]:
df.drop(columns=["distance_m",'duration_sec','duration_in_traffic_sec']).to_excel(OUTPUT_CSV_PATH, index=False)